In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import os

import numpy as np

from torch_model import TextRNN, TextCNN
from cnews_loader import read_vocab, read_category, batch_iter, process_file, build_vocab

2023-12-11 17:29:36.390132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 17:29:36.555300: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 17:29:36.559090: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr

In [1]:
import torch
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

x = torch.randn(1, 10).to(device)

# module
lin = nn.Linear(10, 10, bias=False).to(device)
out = lin(x)
print(lin)


/home/ma-user/anaconda3/envs/PyTorch-1.4/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Linear(in_features=10, out_features=10, bias=False)


In [2]:
base_dir = 'cnews'
train_dir = os.path.join(base_dir, 'cnews.train.txt')
test_dir = os.path.join(base_dir, 'cnews.test.txt')
val_dir = os.path.join(base_dir, 'cnews.val.txt')
vocab_dir = os.path.join(base_dir, 'cnews.vocab.txt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
def evaluate(model, Loss, x_val, y_val):
    """测试集上准确率评估"""
    batch_val = batch_iter(x_val, y_val, 64)
    acc = 0
    los = 0
    for x_batch, y_batch in batch_val:
        size = len(x_batch)
        x = np.array(x_batch)
        y = np.array(y_batch)
        x = torch.LongTensor(x).to(device)
        y = torch.Tensor(y).to(device)
        # y = torch.LongTensor(y)
        # x = Variable(x)
        # y = Variable(y)
        out = model(x)
        loss = Loss(out, y)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        loss_value = np.mean(loss.cpu().detach().numpy())
        accracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
        acc +=accracy*size
        los +=loss_value*size
    return los/len(x_val), acc/len(x_val)

In [18]:
def train():
    x_train, y_train = process_file(train_dir, word_to_id, cat_to_id,600)#获取训练数据每个字的id和对应标签的oe-hot形式
    x_val, y_val = process_file(val_dir, word_to_id, cat_to_id,600)
    #使用LSTM或者CNN
    model = TextRNN()
    
    model.to(device)
    # model = TextCNN()
    
    #选择损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    # Loss = nn.BCELoss()
    # Loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(),lr=0.001)
    best_val_acc = 0
    for epoch in range(20):
        i = 0
        print('epoch:{}'.format(epoch))
        batch_train = batch_iter(x_train, y_train,64)
        for x_batch, y_batch in batch_train:
            i +=1
            # print(i)
            x = np.array(x_batch)
            y = np.array(y_batch)
            x = torch.LongTensor(x).to(device)
            y = torch.Tensor(y).to(device)
            # y = torch.LongTensor(y)
            # x = Variable(x)
            # y = Variable(y)
            out = model(x)
            loss = Loss(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 对模型进行验证
            if i % 90 == 0:
                los, accracy = evaluate(model, Loss, x_val, y_val)
                print('loss:{},accracy:{}'.format(los, accracy))
                if accracy > best_val_acc:
                    torch.save(model.state_dict(), 'model_params.pkl')
                    best_val_acc = accracy

In [20]:
#获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
#获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab(vocab_dir)
#获取字数
vocab_size = len(words)
print('train')
train()

train
epoch:0


/home/ma-user/anaconda3/envs/PyTorch-1.4/lib/python3.7/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss:0.7233162692070008,accracy:0.2612
loss:0.718931548500061,accracy:0.3282
loss:0.7176477731704712,accracy:0.3378
loss:0.7192708904266357,accracy:0.326
loss:0.7134006710052491,accracy:0.3968
loss:0.7138477840423584,accracy:0.3802
loss:0.7150819551467895,accracy:0.3692
loss:0.7125554258346558,accracy:0.4092
epoch:1
loss:0.7114644678115845,accracy:0.4132
loss:0.7093504576683044,accracy:0.4318
loss:0.710750887966156,accracy:0.4176
loss:0.7125651340484619,accracy:0.409
loss:0.7235299991607667,accracy:0.2756
loss:0.7187762597084045,accracy:0.3322
loss:0.7172029609680176,accracy:0.3538
loss:0.7128728792190552,accracy:0.403
epoch:2
loss:0.7116015760421753,accracy:0.4178
loss:0.710459390258789,accracy:0.428
loss:0.7096115100860596,accracy:0.4424
loss:0.711233701992035,accracy:0.4314
loss:0.7059494918823243,accracy:0.4812
loss:0.7041563882827758,accracy:0.5056
loss:0.7035351189613342,accracy:0.5106
loss:0.7010683180809021,accracy:0.538
epoch:3
loss:0.7320642203330994,accracy:0.2034
loss:0.722

KeyboardInterrupt: 